# Wind Farm Spotter: Inference Test (Single) [Python, CPU Runtime]

### **Environment Setup**
First setting up the colab/jupyter notebook with necessary packages; also mounting Google Drive for our external working directory/file store.

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Install pytorchcv** package to have access to base architecture implementations.

In [0]:
!pip install pytorchcv

**Imports:**

In [0]:
import fastai
from fastai.vision import *
from pytorchcv.model_provider import get_model as ptcv_get_model
import warnings
warnings.filterwarnings('ignore')
print(fastai.__version__)

1.0.59


In [0]:
path = Path("/content/drive/My Drive/dla-project/data/")
path.ls()

[PosixPath('/content/drive/My Drive/dla-project/data/test'),
 PosixPath('/content/drive/My Drive/dla-project/data/train'),
 PosixPath('/content/drive/My Drive/dla-project/data/test_single'),
 PosixPath('/content/drive/My Drive/dla-project/data/effnet-b1-c9-sd'),
 PosixPath('/content/drive/My Drive/dla-project/data/effnet-b1-sd.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/incept-v3-80-sd.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd1.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd2.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd3.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd4.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd5.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd6.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd7.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept

### **Reloading the Fast.ai Learner**
Fast.ai provides a useful method named **load_learner()** to reinstantiate an instance model from a pickle file. The loaded learner recalls all the configurations(e.g., transformations, batch size, etc.) used during training. load_learner() also gives us the option of passing a list of images (unlabeled) right from the start. To run our Single Image Inference Test, we'll use the single_test directory to create a Fast.ai **ImageList**..

In [0]:
inference_effnet_single = load_learner(path, file=path/'train/effnet-b1-c98.pkl', test=ImageList.from_folder(path/"test_single"))
inference_incept_single = load_learner(path, file=path/'train/incept-v3-95-2.pkl', test=ImageList.from_folder(path/"test_single"))


**Create a name list:**

In [0]:
test_files = path/"test_single"
test_name_list = os.listdir(test_files)
#test_name_list[:5]

**EfficientNet Single Inference:**

In [0]:
gc.collect()

7

In [0]:
import time

Here, we set our model to evaluation mode...

In [0]:
inference_effnet_single.model.eval()

and loop through the learner's test dataset, making a prediction for each item, and storing the result in our eff_preds dictionary: 

In [0]:
eff_preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_effnet_single.data.test_dl.dl.dataset.x[i]
  p = inference_effnet_single.predict(img)
  eff_preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(eff_preds.items())
print("elapsed: ", end - start)

elapsed:  32.28837037086487


**Results:**

In [0]:
results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_med_potential
3                test_high_capacity_3.jpg      turbines_high_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg      turbines_high_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg   no_turbines_low_potential
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_capacity
13                te

**Inception-v3 Single Inference:**

We'll do the same for Inception-v3:


In [0]:
gc.collect()

0

In [0]:
inference_incept_single.model.eval()

In [0]:
incept_preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_incept_single.data.test_dl.dl.dataset.x[i]
  p = inference_incept_single.predict(img)
  incept_preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(incept_preds.items())
print("elapsed: ", end - start)

elapsed:  32.21531343460083


Based on these findings, it appears that CPU inference takes double the time of GPU Inference (Tesla P100-PCIE-16GB).

**Results:**

In [0]:
results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg    turbines_medium_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_low_potential
3                test_high_capacity_3.jpg    turbines_medium_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg    turbines_medium_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg       turbines_low_capacity
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_capacity
13                te